In [1]:
import pickle as pkl
import pandas as pd
import matplotlib
import os
import re

import collections
import datetime
import time

import geopandas as gpd

import numpy as np

import rasterio

import spacy

import gc

import geopy.distance

nlp = spacy.load('en_core_web_lg')
from sklearn.metrics.pairwise import cosine_similarity

import itertools

RuntimeError: Failed to import transformers.models.auto because of the following error (look up to see its traceback):
Failed to import transformers.modeling_tf_utils because of the following error (look up to see its traceback):
dlopen(/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/_pywrap_tfe.so, 0x0002): Library not loaded: @rpath/_pywrap_tensorflow_internal.so
  Referenced from: /Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/_pywrap_tfe.so
  Reason: tried: '/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/../../_solib_darwin_x86_64/_U_S_Stensorflow_Spython_C_Upywrap_Utfe.so___Utensorflow/_pywrap_tensorflow_internal.so' (no such file), '/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/../../_solib_darwin_x86_64/_U_S_Stensorflow_Spython_C_Upywrap_Utensorflow_Uinternal_Umacos___Utensorflow_Spython/_pywrap_tensorflow_internal.so' (no such file), '/Users/brianreed/opt/anac

# Compustat and ABI Linking

In [ ]:
gvKey_abiLinkingTable = pd.read_csv('../../data/companyData/linkingTable.csv').drop(columns = ['Unnamed: 0'])

base_columns = gvKey_abiLinkingTable.columns 
customer_columns = "customer_" + base_columns
supplier_columns = "supplier_" + base_columns



hasMatch = gvKey_abiLinkingTable.gvkey.unique()

gvKey_abiLinkingTable.head()


In [ ]:
lines = gvKey_abiLinkingTable.abi.astype('str').unique()

print(lines)

with open('../../data/companyData/igCompanies.txt', 'w') as f:
    for line in lines:
        f.write(line)
        f.write('\n')

---------------------------------

# Get all change data together
Get the linking table and merge the abi labels into the change df. 

Then, merge the location data into the change data and get as complete a record of companies as possible given the HQ data.

In [2]:
gvKey_abiLinkingTable = pd.read_csv('../../data/companyData/linkingTable.csv').drop(columns = ['Unnamed: 0'])

changes = pd.read_csv("../../data/companyData/compustatChanges_withControls.csv").drop(columns = ['Unnamed: 0'])
print(changes.shape, changes.head())


changesABI = changes.merge(gvKey_abiLinkingTable, on ='gvkey').drop(columns = {'state','city'})
print(changesABI.shape, changesABI.head())

(649614, 59)    gvkey  datadate  year  qtr              companyName curcdq      assets  \
0   8515  19990930  1999    3                  PHI INC    USD         NaN   
1  12405  19990930  1999    3               AVESIS INC    USD         NaN   
2  24474  19990930  1999    3  TECHNOLOGY SOLUTIONS CO    USD  362.960666   
3  26830  19990930  1999    3   NAMIBIAN MINERALS CORP    USD  111.299129   
4  30448  19990331  1970    1   AMERICAN HOMESTAR CORP    USD         NaN   

   cash  costGoodsSold  totalInv  ...  sic2            indGroup  earliestYear  \
0   NaN            NaN       NaN  ...    45  transportUtilities          1981   
1   NaN       2.470251       NaN  ...    64             finance          1984   
2   NaN      36.229854  0.000000  ...    73            services          1990   
3   NaN       7.938061  4.269385  ...    14              mining          1994   
4   NaN            NaN       NaN  ...    24                manu          1993   

   ageTercile sizeTercile  profitTerc

Now merge in the hq information.

In [3]:
canadian = ['ON', 'AB','QC', 'BC', 'NS', 'NF', 'SK', 'MB', 'NB']
changes = changes[~(changes.state.isin(canadian)) & ~changes.state.isna()]

changes['addzip'] = changes.addzip.astype('str').str.slice(0,5)

changes.state.unique()

array(['LA', 'AZ', 'IL', 'TX', 'IA', 'GA', 'CA', 'PA', 'MO', 'SC', 'NC',
       'UT', 'NV', 'MA', 'MN', 'OH', 'CO', 'FL', 'NJ', 'NY', 'WI', 'IN',
       'VA', 'MD', 'CT', 'MI', 'KS', 'TN', 'DE', 'WA', 'OR', 'HI', 'KY',
       'RI', 'ME', 'NM', 'VT', 'AL', 'AR', 'ID', 'WY', 'OK', 'PR', 'NE',
       'SD', 'DC', 'WV', 'MS', 'ND', 'MT', 'NH', 'AK', 'GU', 'VI'],
      dtype=object)

In [4]:
changesABI.head()

,gvkey,datadate,year,qtr,companyName,curcdq,assets,cash,costGoodsSold,totalInv,...,sizeTercile,profitTercile,datacqtr,datafqtr,fyr,DATE,cstatCompanies,igCompanies,delete,abi
0,8515,19990930,1999,3,PHI INC,USD,NaN,NaN,NaN,NaN,...,1.0,1.0,1999Q3,1999Q3,12,1970-01-01 00:00:00.019990930,phi,phi,NaN,4103925
1,8515,19991231,1999,4,PHI INC,USD,332.308866,NaN,NaN,55.535281,...,1.0,2.0,1999Q4,1999Q4,12,1970-01-01 00:00:00.019991231,phi,phi,NaN,4103925
2,8515,20000331,2000,1,PHI INC,USD,330.571463,NaN,68.550922,56.906485,...,1.0,1.0,2000Q1,2000Q1,12,1970-01-01 00:00:00.020000331,phi,phi,NaN,4103925
3,8515,20000630,2000,2,PHI INC,USD,314.349477,NaN,72.109204,57.186407,...,1.0,1.0,2000Q2,2000Q2,12,1970-01-01 00:00:00.020000630,phi,phi,NaN,4103925
4,8515,20000930,2000,3,PHI INC,USD,315.826957,NaN,76.488119,60.317248,...,1.0,1.0,2000Q3,2000Q3,12,1970-01-01 00:00:00.020000930,phi,phi,NaN,4103925


In [5]:
hq = pd.read_csv("../../data/ig_uniqueHQs_multLocations.csv").\
    drop(columns = {'Unnamed: 0'}).\
    rename(columns = {'archive_version_year': 'year'})

hq['year'] = hq.year.astype('int64')

igChanges = changesABI.merge(hq)
print(igChanges.shape, igChanges.head())


hq.head()

(271847, 69)    gvkey  datadate  year  qtr companyName curcdq      assets  cash  \
0   8515  19990930  1999    3     PHI INC    USD         NaN   NaN   
1   8515  19991231  1999    4     PHI INC    USD  332.308866   NaN   
2   8515  20000331  2000    1     PHI INC    USD  330.571463   NaN   
3   8515  20000630  2000    2     PHI INC    USD  314.349477   NaN   
4   8515  20000930  2000    3     PHI INC    USD  315.826957   NaN   

   costGoodsSold   totalInv  ...  delete      abi  ticker  \
0            NaN        NaN  ...     NaN  4103925    PHII   
1            NaN  55.535281  ...     NaN  4103925    PHII   
2      68.550922  56.906485  ...     NaN  4103925    PHII   
3      72.109204  57.186407  ...     NaN  4103925    PHII   
4      76.488119  60.317248  ...     NaN  4103925    PHII   

                     company state      city address_line_1 zipcode  latitude  \
0  PETROLEUM HELICOPTERS INC    LA  METAIRIE     PO BOX 578   70004  29.97589   
1  PETROLEUM HELICOPTERS INC    LA  M

,abi,ticker,company,year,state,city,address_line_1,zipcode,latitude,longitude
0,7609,SODI,SOLITRON DEVICES INC,1998,FL,WEST PALM BEACH,3301 ELECTRONICS WAY # C,33407,26.74120,-80.06694
1,15578,NaN,BRIDGEPORT MACHINES INC,1998,CT,BRIDGEPORT,500 LINDLEY ST,6606,41.19809,-73.19549
2,23077,NaN,JENNY LEE BAKERY,1998,PA,MC KEES ROCKS,620 ISLAND AVE,15136,40.47235,-80.06152
3,76547,NaN,MASTER PROTECTION CORP,1998,CA,SANTA MONICA,520 BROADWAY # 650,90401,34.01618,-118.49206
4,77743,NaN,NATIONAL TECHNICAL SYSTEMS INC,1998,CA,CALABASAS,24007 VENTURA BLVD # 200,91302,34.15562,-118.65163


In [30]:
igChanges.year.min()

1999

In [31]:
igChanges.to_csv("../../data/companyData/igData.csv")

In [32]:
igChanges.columns

Index(['gvkey', 'datadate', 'year', 'qtr', 'companyName', 'curcdq', 'assets',
       'cash', 'costGoodsSold', 'totalInv', 'netIncome', 'opInc_afDep',
       'opInc_befDep', 'totalRevenue', 'costat', 'priceClose', 'add1',
       'addzip', 'assetsLast', 'netIncomeLast', 'totalRevenueLast',
       'costGoodsSoldLast', 'totalInvLast', 'opInc_afDepLast',
       'opInc_befDepLast', 'priceCloseLast', 'cashLast', 'fyearq',
       'assetsLagged', 'netIncomeLagged', 'roa_lagged', 'sic2', 'indGroup',
       'earliestYear', 'ageTercile', 'sizeTercile', 'profitTercile',
       'datacqtr', 'datafqtr', 'fyr', 'DATE', 'cstatCompanies', 'igCompanies',
       'delete', 'abi', 'ticker', 'company', 'state', 'city', 'address_line_1',
       'zipcode', 'latitude', 'longitude'],
      dtype='object')

In [34]:
igChanges.shape

(271281, 53)

At this point, we have zip information in the following forms (from most to least examples):
    - changes: all compustat companies, from the compustat address system
    - igChanges: subset of compustat companies, from the ig merge
    - subset of compustat companies that have SC information and survived the ig merge
    
We could potentially look at the subset of compustat companies for which we have SC information, usign the compustat address system as well.

For now: follow similar trajectory as before but add in weather data for all cstat companies and all ig-merged companies.

First: pull all zips that are mentioned in changes and igChanges and use this to get the weather data.



In [6]:
changes = changes[(~changes.addzip.isna()) & (changes.addzip != 'nan')]
relevantZips = changes.addzip.astype('int64').append(igChanges.zipcode).unique()

changes.rename(columns = {'addzip': 'zipcode'}, inplace = True)
changes.drop(columns = {'datadate','costat', 'add1', 'city',  'state'}, inplace = True)

# 

In [10]:
len(relevantZips)

5048

In [11]:
# relevantZips = allCustomerData.zipcode.append(allSupplierData.zipcode).unique()
outfile =  '../../data/companyData/relevantZips.pkl'
with open(outfile, 'wb') as pickle_file:
    pkl.dump(relevantZips, pickle_file)

------------------------------------------------

## create the original weather with lags dataset

In [8]:
'''g = pd.read_csv("../../data/companyData/weatherByEstablishment.csv").\
    drop(columns = {"Unnamed: 0"})'''


allWeather_withLags = pd.read_csv("../../data/companyData/allWeather_withLags_allZips.csv").\
    drop(columns = {"Unnamed: 0", 'yearQtr'}).astype({'year':       'category',
                           'qtr':        'category',
                           'zipcode':    'category'})


averages = pd.read_csv("../../data/companyData/quarterlyStatsByZip.csv").\
    drop(columns = {"Unnamed: 0"}).rename(columns = {'ZIP': 'zipcode'})
averages['qtr'] = averages.quarter.str.slice(1,2).astype('float')
averages.drop(columns = {'quarter'}, inplace = True) 
averages = averages.astype({'qtr':        'category',
                           'zipcode':    'category'})


allWeather_withLags2 = pd.read_csv("../../data/companyData/allWeather_withLags_new.csv").\
    drop(columns = {"Unnamed: 0", 'yearQtr'}).astype({'year':       'category',
                           'qtr':        'category',
                           'zipcode':    'category'})


'''thunderstorms_withLags = pd.read_csv("../../data/companyData/thunderstorms_withLags.csv").\
    drop(columns = {"Unnamed: 0", 'yearQtr'}).astype({'year':       'category',
                           'qtr':        'category',
                           'zipcode':    'category'})'''

'thunderstorms_withLags = pd.read_csv("../../data/companyData/thunderstorms_withLags.csv").    drop(columns = {"Unnamed: 0", \'yearQtr\'}).astype({\'year\':       \'category\',\n                           \'qtr\':        \'category\',\n                           \'zipcode\':    \'category\'})'

Create direct effects database. Merge weather to full cstat database.

Merge weather to the ig-cstat database.

In [9]:
igChanges = pd.read_csv("../../data/companyData/igData.csv").drop(columns = {'Unnamed: 0'})

'''fractions_byZip = pd.read_csv("../../data/companyData/fractions_byZip.csv").drop(columns = {'Unnamed: 0'})
fractions_byZip = fractions_byZip[fractions_byZip.gvkey.isin(list(igChanges.gvkey.unique())) & \
                                  fractions_byZip.zipcode.isin(list(igChanges.zipcode.unique())) ]

fractions_byZip = fractions_byZip.groupby(['year','zipcode','gvkey']).sum().reset_index()

print(igChanges.shape)

igChanges = igChanges.merge(fractions_byZip)'''

print(igChanges.shape)

(271847, 69)


In [10]:
igChanges.year.min()

1999

In [12]:
igChangesWithWeather = igChanges.merge(allWeather_withLags).merge(allWeather_withLags2).\
    merge(averages)#.merge(thunderstorms_withLags) .merge(g)
igChangesWithWeather.shape # 239068

(239599, 215)

In [13]:
for col in igChangesWithWeather.columns:
    print(col) 

gvkey
datadate
year
qtr
companyName
curcdq
assets
cash
costGoodsSold
totalInv
netIncome
opInc_afDep
opInc_befDep
totalRevenue
costat
priceClose
add1
addzip
earningsPerShare
sales
otherCosts
shares
assetsLast
netIncomeLast
totalRevenueLast
costGoodsSoldLast
totalInvLast
opInc_afDepLast
opInc_befDepLast
priceCloseLast
cashLast
earningsPerShareLast
sharesLast
salesLast
otherCostsLast
incomeChange
revenueChange
costChange
inventoryChange
opInc_afDepChange
opInc_befDepChange
priceCloseChange
assetsPrev
fyearq
assetsLagged
netIncomeLagged
roa_lagged
sic2
indGroup
earliestYear
ageTercile
sizeTercile
profitTercile
datacqtr
datafqtr
fyr
DATE
cstatCompanies
igCompanies
delete
abi
ticker
company
state
city
address_line_1
zipcode
latitude
longitude
precip_annual_50
precip_annual_95
precip_annual_99
precip_zip_50
precip_zip_95
precip_zip_99
precip_zipQuarter_50
precip_zipQuarter_95
precip_zipQuarter_99
temp_annual_50
temp_annual_95
temp_annual_99
temp_zip_50
temp_zip_95
temp_zip_99
temp_zipQuarter_

In [30]:
igChangesWithWeather.gvkey

0           8515
1          25874
2          28564
3           8515
4          25874
           ...  
239594    112159
239595    116675
239596    116675
239597    116675
239598    116675
Name: gvkey, Length: 239599, dtype: int64

In [38]:
fractions.year

0          2003
1          2003
2          2003
3          2003
4          2003
           ... 
1557409    2002
1557410    2002
1557411    2002
1557412    2002
1557413    2002
Name: year, Length: 1557413, dtype: object

In [50]:
fractions = pd.read_csv("../../data/companyData/fractionEmployees_byEstablishment.csv").\
    drop(columns = ['latitude', 'longitude']).\
    rename(columns = {'archive_version_year': 'year',
                     'parent_number': 'gvkey'})

fractions = fractions[fractions.gvkey != '/']

fractions['year'] = fractions['year'].astype('int64')
fractions['gvkey'] = fractions['gvkey'].astype('int64')
fractions['zipcode'] = fractions['zipcode'].astype('int64')

igChangesWithWeather['year'] = igChangesWithWeather['year'].astype('int64')
igChangesWithWeather['zipcode'] = igChangesWithWeather['zipcode'].astype('int64')

print(fractions.head())


merged_dataframe = pd.merge_asof(igChangesWithWeather.sort_values('year'), fractions.sort_values('year'), on="year",
                                 by= ['gvkey','year','zipcode'], 
                                direction = "nearest")

/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


   year  gvkey  zipcode  locationFracOfEmployees
0  2003  23077    15222                 0.147059
1  2003  23077    15136                 0.735294
2  2003  76547    33907                 0.121951
3  2003  76547    75062                 0.104530
4  2003  76547    92131                 0.104530


In [55]:
igChangesWithWeather.year

0         2003
1         2003
2         2003
3         2004
4         2004
          ... 
239594    2019
239595    2019
239596    2019
239597    2019
239598    2019
Name: year, Length: 239599, dtype: int64

In [14]:
igChangesWithWeather.to_csv("../../data/companyData/hqsOnly/igWithWeather.csv")

In [22]:
igChangesWithWeather.shape

(239599, 215)

# Supplier Focus
Subset to focus on firms who are listed as suppliers, in the years of focus.

In [3]:
igChangesWithWeather = pd.read_csv("../../data/companyData/hqsOnly/igWithWeather.csv").drop(columns = 'Unnamed: 0')

In [15]:
suppliersOnly = pd.read_csv("../../data/companyData/suppliers.csv").drop(columns = 'Unnamed: 0').\
    rename(columns = {'supplier_gvkey': 'gvkey'})

In [16]:
igChangesWithWeather.columns

Index(['gvkey', 'datadate', 'year', 'qtr', 'companyName', 'curcdq', 'assets',
       'cash', 'costGoodsSold', 'totalInv',
       ...
       'lag4_precip_zipWeek50', 'lag4_precip_zipWeek95', 'lag4_days90Plus',
       'lag4_streak90Plus', 'quarterly_avg_precip', 'quarterly_median_precip',
       'quarterly_variance_precip', 'quarterly_avg_temp',
       'quarterly_median_temp', 'quarterly_variance_temp'],
      dtype='object', length=215)

In [17]:
suppliersOnly.columns

Index(['year', 'gvkey', 'supplierYear'], dtype='object')

In [18]:
allSupplierData = suppliersOnly.merge(igChangesWithWeather)

In [19]:
allSupplierData.shape

(66277, 216)

In [20]:
allSupplierData.head()

,year,gvkey,supplierYear,datadate,qtr,companyName,curcdq,assets,cash,costGoodsSold,...,lag4_precip_zipWeek50,lag4_precip_zipWeek95,lag4_days90Plus,lag4_streak90Plus,quarterly_avg_precip,quarterly_median_precip,quarterly_variance_precip,quarterly_avg_temp,quarterly_median_temp,quarterly_variance_temp
0,2009,1013.0,1013_2009,20091231,4,ADC TELECOMMUNICATIONS INC,USD,1600.631743,669.076288,189.162302,...,8,0,0,0,1.431738,0.0,24.032696,5.485464,4.858000,94.337883
1,2010,1013.0,1013_2010,20100331,1,ADC TELECOMMUNICATIONS INC,USD,1603.590912,531.947264,188.959304,...,8,1,0,0,0.953737,0.0,8.152314,-0.983637,-0.383000,62.634399
2,2010,1013.0,1013_2010,20100630,2,ADC TELECOMMUNICATIONS INC,USD,1681.842764,533.910791,209.428053,...,4,0,4,0,3.057708,0.0,52.537414,19.777313,20.761499,58.708854
3,2010,1013.0,1013_2010,20100930,3,ADC TELECOMMUNICATIONS INC,USD,1742.356995,612.217809,218.134351,...,5,0,0,0,3.428374,0.0,90.593667,25.752763,26.467500,26.154579
4,2004,1050.0,1050_2004,20040331,1,CECO ENVIRONMENTAL CORP,USD,53.955953,NaN,15.469965,...,6,2,0,0,2.703528,0.0,42.561597,7.481237,7.095000,57.686246


In [21]:
allSupplierData.to_csv("../../data/companyData/hqsOnly/allSupplierData.csv")